### Context ###

https://dynomight.net/air/

### Input data ###

In [1]:
# Количество растворённых твёрдых веществ ("total dissolved solids") в используемой воде, ppm (parts per million, единиц на миллион)
# из под крана - 100. после обратного осмоса - 15. минеральная вода из бутылки - 100.
water_tds_ppm = 15

# параметры воздуха
air_temperature_celsius = 22  # (°C)
air_humidity_start_pct = 22  # относительная влажность (%) в начале
air_humidity_end_pct = 42  # относительная влажность (%) в конце
air_pressure_kpa = 100  # атмосферное давление (килопаскаль); 1 атм. = 101.325 кПа

### Calculation ###

In [15]:
import math
from IPython.display import display_markdown

air_temperature_kelvin = air_temperature_celsius + 273.15
air_density_g_per_m3 = (
    4.81e-26  #  molecular mass of dry air
    / 1.380649e-23  # Boltzmann constant
    * 1000  # kg to g
    * 1000  # kPa to Pa
    * air_pressure_kpa / air_temperature_kelvin
)
display_markdown("### Плотность воздуха, грамм на метр кубический ###", raw=True)
display_markdown(f"{air_density_g_per_m3=}", raw=True)

# Reference: https://www.aqua-calc.com/calculate/humidity

# Saturated water vapor pressure (Pws)
# https://en.wikipedia.org/wiki/Vapour_pressure_of_water
# "Tetens equation" (simple)
pws_by_tetens_kpa = 0.61078 * math.exp(17.27 * air_temperature_celsius / (air_temperature_celsius + 237.3))
# ln(Pws ⁄ Pc) = Tc ⁄ T × (a1 × ϑ + a2 × ϑ^1.5 + a3 × ϑ^3 + a4 × ϑ^3.5+ a5 × ϑ^4 + a6 × ϑ^7.5)
# where:  Pws  is saturated water vapor pressure,  Pc  is critical pressure equal to 22.064 MPa,
# Tc is critical temperature equal to 647.096 K,  ϑ = (1 − T ⁄ Tc)
_t_critical_kelvin = 647.096
_pws_theta = 1 - air_temperature_kelvin / _t_critical_kelvin
pws_kpa = 22.064 * 1000 * math.exp(
    _t_critical_kelvin / air_temperature_kelvin * (
        -7.85951783 * _pws_theta + 1.84408259 * _pws_theta**1.5 + -11.7866497 * _pws_theta**3
        + 22.6807411 * _pws_theta**3.5 + -15.9618719 * _pws_theta**4 + 1.80122502 * _pws_theta**7.5)
)

# Saturated mixing ratio (Ws):  Ws, g/kg = 621.97 × Pws ⁄ (P − Pws)
# where  P is ambient pressure,  Pws  is saturated water vapor pressure.  [5]
water_saturated_mixing_ratio = 621.97 / 1000 * pws_kpa / (air_pressure_kpa - pws_kpa)
display_markdown("### Доля воды в воздухе на 100% влажности ###", raw=True)
display_markdown(f"{water_saturated_mixing_ratio=}", raw=True)

added_water_g_per_m3 = (
    (air_humidity_end_pct - air_humidity_start_pct) / 100
    * air_density_g_per_m3 * water_saturated_mixing_ratio
)
display_markdown("### Масса добавленой воды, грамм на метр кубический ###", raw=True)
display_markdown(f"{added_water_g_per_m3=}")
display_markdown(f"с {air_humidity_start_pct}% до {air_humidity_end_pct}% влажности", raw=True)

added_solids_ug_per_m3 = (
    added_water_g_per_m3
    * (water_tds_ppm / 1e6)
    * 1e6  # gram -> microgram
)
display_markdown("### Масса добавленой пыли, микрограмм на метр кубический ###", raw=True)
display_markdown(f"{added_solids_ug_per_m3=}", raw=True)


### Плотность воздуха, грамм на метр кубический ###

air_density_g_per_m3=1180.3722914501784

### Доля воды в воздухе на 100% влажности ###

water_saturated_mixing_ratio=0.016899524415714722

### Масса добавленой воды, грамм на метр кубический ###

с 22% до 42% влажности

### Масса добавленой пыли, микрограмм на метр кубический ###

added_solids_ug_per_m3=59.84319107698628